# **ETL**

---

### Extracción de datos

In [46]:
# Librerias a utilizar
import requests
import pandas as pd
import calendar
import io
import datetime
import json

In [297]:
# Extraemos los datos mediante una API 

# Creamos lista donde almacenaremos todos los datos
all_data = []

# Iteramos para extraer los datos de los años de interes, en este caso de 1950 a 2023
for i in range(1970, 2023):
    year = i
    for mes in range (1, 13):
        # Tomamos el último dia de cada mes
        last_day = calendar.monthrange(year, mes)[1]
        starttime = f"{year}-{mes}-01"
        endtime = f"{year}-{mes}-{last_day}"
        # El url de la api con los parámetros, donde filtramos buscando magnitudes mayores a 4
        url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={starttime}&endtime={endtime}&minmagnitude=4"
        # Envía la solicitud HTTP a la API de USGS y procesa los datos de respuesta
        response = requests.get(url)
        data = json.loads(response.text)
        data = data["features"]
                # Itera sobre cada objeto "Feature" en la lista "data"
        for feature in data:
                # Accede a la parte "properties" de cada objeto
            properties = feature['properties']
            properties['year'] = year
            properties['month'] = mes

            all_data.append(properties)
    # Crea un dataframe de Pandas a partir de la lista de eventos sísmicos
    
            

# Realizamos la misma extracción, pero para el año 2023 de forma actualizada siempre al tiempo actual
# Tomamos el mes actual
mes_actual = datetime.datetime.now().month
# Iteramos por mes, con tope en el mes actual
for mes in range (1, mes_actual +1):
    # Tomamos el último dia de cada mes
    last_day = calendar.monthrange(2023, mes)[1]
    starttime = f"2023-{mes}-01"
    endtime = f"2023-{mes}-{last_day}"
    url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime={starttime}&endtime={endtime}&minmagnitude=4"
    response = requests.get(url)
    data = json.loads(response.text)
    data = data["features"]
    # Itera sobre cada objeto "Feature" en la lista "data"
    for feature in data:
     # Accede a la parte "properties" de cada objeto
        properties = feature['properties']
        properties['year'] = 2023
        properties['month'] = mes
    # Crea un dataframe de Pandas a partir de la lista de eventos sísmicos
        all_data.append(properties)
            
# Concatenamos todos los dataframes en uno solo
df_total = pd.DataFrame(all_data)

In [313]:
# Observamos
df_total

,mag,place,tz,felt,cdi,mmi,alert,status,tsunami,sig,...,code,types,nst,dmin,rms,gap,magType,type,year,month
0,4.09,"65km ENE of Beatty, NV",NaN,NaN,NaN,NaN,None,reviewed,0,257,...,3324919,",origin,phase-data,",13.0,1.749,0.31,253.0,ml,nuclear explosion,1970,1
1,5.90,"59 km NNE of Port-Olry, Vanuatu",NaN,NaN,NaN,3.894,None,reviewed,0,536,...,800507,",origin,shakemap,",NaN,NaN,NaN,NaN,mw,earthquake,1970,1
2,6.14,Fiji region,NaN,NaN,NaN,NaN,None,reviewed,0,580,...,800471,",origin,",NaN,NaN,NaN,NaN,mw,earthquake,1970,1
3,5.27,"195 km WSW of Linxia Chengguanzhen, China",NaN,NaN,NaN,NaN,None,reviewed,0,427,...,800431,",origin,",NaN,NaN,NaN,NaN,mw,earthquake,1970,1
4,5.77,"38 km SSE of San Cristóbal, Venezuela",NaN,NaN,NaN,6.276,None,reviewed,0,512,...,800429,",origin,shakemap,",NaN,NaN,NaN,NaN,mw,earthquake,1970,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461654,5.30,"82 km ESE of Katsuren-haebaru, Japan",NaN,2.0,2.7,NaN,None,reviewed,0,433,...,7000jxaa,",dyfi,internal-moment-tensor,moment-tensor,ori...",74.0,0.909,0.83,64.0,mww,earthquake,2023,5
461655,5.10,"Ryukyu Islands, Japan",NaN,NaN,NaN,NaN,None,reviewed,0,400,...,7000jxab,",origin,phase-data,",77.0,0.888,1.17,64.0,mb,earthquake,2023,5
461656,4.30,"45 km NNE of Klyuchi, Russia",NaN,NaN,NaN,NaN,None,reviewed,0,284,...,7000jxa6,",origin,phase-data,",40.0,3.949,0.63,126.0,mb,earthquake,2023,5
461657,5.10,None,NaN,NaN,NaN,NaN,None,reviewed,0,400,...,7000jx9j,",origin,phase-data,",66.0,0.682,0.59,121.0,mww,earthquake,2023,5


In [299]:
# Observamos las columnas, hay algunas que no aportan nada de información útil para el análisis
df_total.columns

Index(['mag', 'place', 'time', 'updated', 'tz', 'url', 'detail', 'felt', 'cdi',
       'mmi', 'alert', 'status', 'tsunami', 'sig', 'net', 'code', 'ids',
       'sources', 'types', 'nst', 'dmin', 'rms', 'gap', 'magType', 'type',
       'title', 'year', 'month'],
      dtype='object')

In [300]:
# Eliminamos las columnas que no utilizaremos
df_total.drop(['time', 'updated',"url","detail","ids","sources","title"],inplace=True,axis=1)

In [312]:
# Observamos
df_total.tail()

,mag,place,tz,felt,cdi,mmi,alert,status,tsunami,sig,...,code,types,nst,dmin,rms,gap,magType,type,year,month
461654,5.3,"82 km ESE of Katsuren-haebaru, Japan",NaN,2.0,2.7,NaN,None,reviewed,0,433,...,7000jxaa,",dyfi,internal-moment-tensor,moment-tensor,ori...",74.0,0.909,0.83,64.0,mww,earthquake,2023,5
461655,5.1,"Ryukyu Islands, Japan",NaN,NaN,NaN,NaN,None,reviewed,0,400,...,7000jxab,",origin,phase-data,",77.0,0.888,1.17,64.0,mb,earthquake,2023,5
461656,4.3,"45 km NNE of Klyuchi, Russia",NaN,NaN,NaN,NaN,None,reviewed,0,284,...,7000jxa6,",origin,phase-data,",40.0,3.949,0.63,126.0,mb,earthquake,2023,5
461657,5.1,None,NaN,NaN,NaN,NaN,None,reviewed,0,400,...,7000jx9j,",origin,phase-data,",66.0,0.682,0.59,121.0,mww,earthquake,2023,5
461658,4.2,"66 km NNE of Bandar Abbas, Iran",NaN,NaN,NaN,NaN,None,reviewed,0,271,...,7000jx9i,",origin,phase-data,",48.0,2.770,0.75,143.0,mb,earthquake,2023,5


In [337]:
# Exportamos
df_total.to_json("df_total")

---

# USA

In [303]:
estados2 = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',
       'Hawai', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey',
       'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio',
       'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island',
       'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah',
       'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin',
       'Wyoming']


In [315]:
df_total["place"].isna().sum()

0

In [314]:
df_total["place"].fillna("sin dato",inplace=True)

In [316]:
all_dfs2 = []
for i in estados2 : 
    all_dfs2.append(df_total[df_total["place"].str.contains(i)])

df_usa = pd.concat(all_dfs2, ignore_index=True)   

In [317]:
df_usa

,mag,place,tz,felt,cdi,mmi,alert,status,tsunami,sig,...,code,types,nst,dmin,rms,gap,magType,type,year,month
0,4.5,"3 km NNW of Fayette, Alabama",NaN,NaN,NaN,5.447,None,reviewed,0,312,...,p0000brg,",origin,shakemap,trump-shakemap,",NaN,NaN,NaN,NaN,mb,earthquake,1975,6
1,4.2,"4 km SW of Brookwood, Alabama",NaN,NaN,NaN,NaN,None,reviewed,0,271,...,p0002tkj,",impact-text,origin,phase-data,",NaN,NaN,0.80,NaN,mb,rock burst,1986,5
2,4.8,"15 km NNW of Flomaton, Alabama",NaN,NaN,NaN,6.012,None,reviewed,0,354,...,p00089hq,",impact-text,origin,phase-data,shakemap,trump-...",NaN,NaN,0.98,NaN,mb,earthquake,1997,10
3,4.8,"14 km WNW of North Johns, Alabama",NaN,NaN,NaN,NaN,None,reviewed,0,354,...,p000918y,",impact-text,origin,phase-data,",NaN,NaN,1.00,NaN,mb,rock burst,1999,1
4,4.6,"8 km S of Valley Head, Alabama",NaN,17204.0,6.0,4.987,green,reviewed,0,926,...,605341,",associate,dyfi,focal-mechanism,impact-text,lo...",21.0,NaN,0.07,154.0,mw,earthquake,2003,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22676,4.0,"23 km SE of Kelly, Wyoming",NaN,153.0,3.4,4.370,green,reviewed,0,298,...,10004t1f,",cap,dyfi,impact-text,losspager,moment-tensor,...",NaN,0.195,1.16,18.0,mwr,earthquake,2016,2
22677,4.8,"4 km ENE of Bondurant, Wyoming",NaN,578.0,3.9,4.640,green,reviewed,0,580,...,20006umx,",cap,dyfi,impact-text,losspager,moment-tensor,...",NaN,0.373,1.14,7.0,mwr,earthquake,2016,8
22678,4.0,"57 km S of Ten Sleep, Wyoming",NaN,58.0,4.1,3.130,green,reviewed,0,270,...,2000bi6d,",dyfi,losspager,moment-tensor,origin,phase-dat...",NaN,1.074,0.74,63.0,mwr,earthquake,2017,11
22679,4.0,"26 km ESE of Wright, Wyoming",NaN,NaN,NaN,NaN,None,reviewed,0,246,...,6000exg9,",origin,phase-data,",NaN,0.984,0.42,54.0,mb,mining explosion,2021,7


In [318]:
df_usa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22681 entries, 0 to 22680
Data columns (total 21 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   mag      22681 non-null  float64
 1   place    22681 non-null  object 
 2   tz       1 non-null      float64
 3   felt     2714 non-null   float64
 4   cdi      2714 non-null   float64
 5   mmi      2733 non-null   float64
 6   alert    1422 non-null   object 
 7   status   22681 non-null  object 
 8   tsunami  22681 non-null  int64  
 9   sig      22681 non-null  int64  
 10  net      22681 non-null  object 
 11  code     22681 non-null  object 
 12  types    22681 non-null  object 
 13  nst      7120 non-null   float64
 14  dmin     5886 non-null   float64
 15  rms      18610 non-null  float64
 16  gap      10151 non-null  float64
 17  magType  22679 non-null  object 
 18  type     22681 non-null  object 
 19  year     22681 non-null  int64  
 20  month    22681 non-null  int64  
dtypes: float64(9

In [336]:
df_usa["place"] = df_usa["place"].str.strip()
df_usa[["place1","place2","place3"]] = df_usa["place"].str.split(",",expand=True)
df_usa

,mag,place,tz,felt,cdi,mmi,alert,status,tsunami,sig,...,dmin,rms,gap,magType,type,year,month,place1,place2,place3
0,4.5,"3 km NNW of Fayette, Alabama",NaN,NaN,NaN,5.447,None,reviewed,0,312,...,NaN,NaN,NaN,mb,earthquake,1975,6,3 km NNW of Fayette,Alabama,None
1,4.2,"4 km SW of Brookwood, Alabama",NaN,NaN,NaN,NaN,None,reviewed,0,271,...,NaN,0.80,NaN,mb,rock burst,1986,5,4 km SW of Brookwood,Alabama,None
2,4.8,"15 km NNW of Flomaton, Alabama",NaN,NaN,NaN,6.012,None,reviewed,0,354,...,NaN,0.98,NaN,mb,earthquake,1997,10,15 km NNW of Flomaton,Alabama,None
3,4.8,"14 km WNW of North Johns, Alabama",NaN,NaN,NaN,NaN,None,reviewed,0,354,...,NaN,1.00,NaN,mb,rock burst,1999,1,14 km WNW of North Johns,Alabama,None
4,4.6,"8 km S of Valley Head, Alabama",NaN,17204.0,6.0,4.987,green,reviewed,0,926,...,NaN,0.07,154.0,mw,earthquake,2003,4,8 km S of Valley Head,Alabama,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22676,4.0,"23 km SE of Kelly, Wyoming",NaN,153.0,3.4,4.370,green,reviewed,0,298,...,0.195,1.16,18.0,mwr,earthquake,2016,2,23 km SE of Kelly,Wyoming,None
22677,4.8,"4 km ENE of Bondurant, Wyoming",NaN,578.0,3.9,4.640,green,reviewed,0,580,...,0.373,1.14,7.0,mwr,earthquake,2016,8,4 km ENE of Bondurant,Wyoming,None
22678,4.0,"57 km S of Ten Sleep, Wyoming",NaN,58.0,4.1,3.130,green,reviewed,0,270,...,1.074,0.74,63.0,mwr,earthquake,2017,11,57 km S of Ten Sleep,Wyoming,None
22679,4.0,"26 km ESE of Wright, Wyoming",NaN,NaN,NaN,NaN,None,reviewed,0,246,...,0.984,0.42,54.0,mb,mining explosion,2021,7,26 km ESE of Wright,Wyoming,None


In [338]:
# Exportamos
df_usa.to_json("Usa.json")

---

# Japón

In [334]:
df_japon = df_total[df_total["place"].str.contains("Japan")]
df_japon

,mag,place,tz,felt,cdi,mmi,alert,status,tsunami,sig,...,code,types,nst,dmin,rms,gap,magType,type,year,month
12,6.40,"45 km SSW of Obihiro, Japan",NaN,NaN,NaN,6.419,None,reviewed,0,630,...,800254,",origin,shakemap,trump-shakemap,",NaN,NaN,NaN,NaN,mw,earthquake,1970,1
77,5.53,"Volcano Islands, Japan region",NaN,NaN,NaN,NaN,None,reviewed,0,470,...,799066,",origin,",NaN,NaN,NaN,NaN,mw,earthquake,1970,2
107,5.54,"67 km NE of Naze, Japan",NaN,NaN,NaN,NaN,None,reviewed,0,472,...,798599,",origin,",NaN,NaN,NaN,NaN,mw,earthquake,1970,3
127,5.70,"101 km WSW of Kurio, Japan",NaN,NaN,NaN,NaN,None,reviewed,0,500,...,798320,",origin,",NaN,NaN,NaN,NaN,mw,earthquake,1970,3
129,5.70,"4 km WSW of Takanosu, Japan",NaN,NaN,NaN,3.951,None,reviewed,0,500,...,798303,",origin,shakemap,",NaN,NaN,NaN,NaN,mw,earthquake,1970,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461650,5.10,"75 km ESE of Katsuren-haebaru, Japan",NaN,NaN,NaN,NaN,None,reviewed,0,400,...,7000jxal,",origin,phase-data,",58.0,0.883,0.99,79.0,mww,earthquake,2023,5
461651,5.00,"81 km SE of Taira, Japan",NaN,NaN,NaN,NaN,None,reviewed,0,385,...,7000jxaj,",origin,phase-data,",44.0,0.845,0.95,78.0,mb,earthquake,2023,5
461652,5.90,"79 km ESE of Katsuren-haebaru, Japan",NaN,10.0,3.1,3.841,green,reviewed,0,539,...,7000jxag,",dyfi,internal-moment-tensor,internal-origin,l...",76.0,0.871,0.75,57.0,mww,earthquake,2023,5
461654,5.30,"82 km ESE of Katsuren-haebaru, Japan",NaN,2.0,2.7,NaN,None,reviewed,0,433,...,7000jxaa,",dyfi,internal-moment-tensor,moment-tensor,ori...",74.0,0.909,0.83,64.0,mww,earthquake,2023,5


In [339]:
df_japon.to_json("Japón.json")

---

# México

In [329]:
# Dataset con magnitudes mayores a 4.0 y de 1970 a 2023
df_mex = pd.read_json("Datasets/sismos_méxico.json")
# Obervamos
df_mex

,Fecha,Hora,Magnitud,Latitud,Longitud,Profundidad,Referencia de localizacion,Fecha UTC,Hora UTC,Estatus
0,1970-02-03,23:08:50,6.6,15.524,-99.493,21.0,"141 km al SUR de SAN MARCOS, GRO",1970-02-04,05:08:50,revisado
1,1970-04-29,08:01:34,7.3,14.463,-92.683,44.0,"62 km al SUROESTE de CD HIDALGO, CHIS",1970-04-29,14:01:34,revisado
2,1970-04-30,02:32:58,6.5,14.608,-93.260,22.0,"100 km al SUROESTE de MAPASTEPEC, CHIS",1970-04-30,08:32:58,revisado
3,1971-09-30,02:18:00,6.5,26.880,-110.800,14.0,"92 km al SUROESTE de PUEBLO YAQUI, SON",1971-09-30,08:18:00,revisado
4,1972-10-20,02:17:46,6.6,18.700,-106.756,10.0,"238 km al OESTE de CIHUATLAN, JAL",1972-10-20,08:17:46,revisado
...,...,...,...,...,...,...,...,...,...,...
42665,2023-05-03,21:17:37,4.0,19.350,-103.870,31.0,"17 km al NOROESTE de VILLA DE ALVAREZ, COL",2023-05-04,03:17:37,verificado
42666,2023-05-03,21:39:31,4.2,16.910,-95.040,106.0,"4 km al NORTE de MATIAS ROMERO, OAX",2023-05-04,03:39:31,verificado
42667,2023-05-04,03:19:06,4.0,15.430,-92.020,212.4,"26 km al NORESTE de MOTOZINTLA, CHIS",2023-05-04,09:19:06,verificado
42668,2023-05-04,04:42:31,4.1,14.880,-94.420,16.1,"151 km al SUROESTE de TONALA, CHIS",2023-05-04,10:42:31,verificado


In [330]:
# Separamos "referencia de localizacion"
df_mex[["Distancia","Estado"]] = df_mex["Referencia de localizacion"].str.split(",",expand=True)
df_mex.sample(3)

,Fecha,Hora,Magnitud,Latitud,Longitud,Profundidad,Referencia de localizacion,Fecha UTC,Hora UTC,Estatus,Distancia,Estado
36479,2020-11-05,23:41:27,4.1,17.3467,-100.691,7.6,"15 km al NOROESTE de TECPAN, GRO",2020-11-06,05:41:27,revisado,15 km al NOROESTE de TECPAN,GRO
37654,2021-04-11,11:20:07,4.9,18.3810,-103.207,52.3,"44 km al SUR de COALCOMAN, MICH",2021-04-11,16:20:07,revisado,44 km al SUR de COALCOMAN,MICH
23149,2017-02-10,21:29:40,4.3,18.4107,-102.591,56.2,"50 km al NOROESTE de LA MIRA, MICH",2017-02-11,03:29:40,revisado,50 km al NOROESTE de LA MIRA,MICH


In [340]:
# Exportamos a formato json
df_mex.to_json("México.json")

---

# Población

In [48]:
# Extraemos los datos via csv
hab_usa = pd.read_csv("Datasets/Habitantes - Censo Usa.csv")
# Ingestamos via CSV la población de México para 2020
hab_mex = pd.read_csv("Datasets/Habitantes - Mexico.csv")

hab_jap = pd.read_csv("Datasets/Habitantes - Japon.csv")


,Estado,Población Estimada (2020)
0,Alabama,"5,024,279"
1,Alaska,"733,391"
2,Arizona,"7,151,502"
3,Arkansas,"3,011,524"
4,California,"39,538,223"


In [49]:
# Colocamos el id
hab_usa["idCountry"] = 1
# Reordenamos
hab_usa = hab_usa[["idCountry","Estado","Población Estimada (2020)"]] 
# Estandarizamos los nombres de columnas
hab_usa.rename(columns={"Población Estimada (2020)": "population", "Estado" : "place"},inplace=True)
# Observamos
hab_usa.head()

In [ ]:
hab_jap["idCountry"] = 2
hab_jap.head(3)
hab_jap["Prefectura y Capital"] = hab_jap["Prefectura"] + "-" + hab_jap["Capital"]
hab_jap.head()
hab_jap.drop(["Prefectura","Capital","Unnamed: 3","Unnamed: 4"],axis=1,inplace=True)
hab_jap.head()
hab_jap = hab_jap[["idCountry","Prefectura y Capital","Población"]]
hab_jap.rename(columns={"Prefectura y Capital" : "place", "Población" : "population"},inplace=True)
hab_jap.head()

In [ ]:
# Realizamos las transformaciones necesarias

# Colocamos el id de México
hab_mex["idCountry"] = 3

# Dropeamos las columnas vacías
hab_mex.drop(["Unnamed: 3","Unnamed: 2"],axis=1,inplace=True)

# Estandarizamos los nombres de las columnas
hab_mex.rename(columns={"Estado" : "place", "Población (2020)" : "population"},inplace=True)

# Reordenamos
hab_mex = hab_mex[["idCountry","place","population"]]

# Obervamos
hab_mex.head()

---